# Example on the MNIST Digits Dataset

### 1. Import of the modules and the dataset

In [1]:
from neural_network import *
from keras.datasets import mnist

2023-08-30 16:46:13.405706: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 2. Load the MNIST dataset

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

### 3. Build the neural network

In [3]:
net = NeuralNetwork(
    Normalization(samples=X_train),
    Reshape(output_shape=(1, 28, 28)),
    Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2),
    BatchNorm2d(6),
    Tanh(),
    AvgPool2d(kernel_size=(2, 2), stride=(2, 2)),
    Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
    BatchNorm2d(16),
    Tanh(),
    AvgPool2d(kernel_size=(2, 2), stride=(2, 2)),
    Flatten(),
    Linear(in_features=400, out_features=120),
    Tanh(),
    Linear(in_features=120, out_features=84),
    Tanh(),
    Linear(in_features=84, out_features=10),
    OutputLayer(activation_function="softmax", loss_function="categorical_cross_entropy")
)
print(net)

NeuralNetwork:
 (0) Normalization(norm=255.0, dtype=float32)
 (1) Reshape(output_shape=(1, 28, 28))
 (2) Conv2d(in_channels=1, out_channels=6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (3) BatchNorm2d(6, eps=1e-05, momentum=0.1)
 (4) Tanh()
 (5) AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
 (6) Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5), stride=(1, 1), padding=(0, 0), optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (7) BatchNorm2d(16, eps=1e-05, momentum=0.1)
 (8) Tanh()
 (9) AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
 (10) Flatten()
 (11) Linear(in_features=400, out_features=120, optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (12) Tanh()
 (13) Linear(in_features=120, out_features=84, optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), ini

### 4. Choose the number of samples to train on

In [4]:
# Select N samples to train on:
N = 3000
X_train, y_train = X_train[:N], y_train[:N]
X_train.shape, y_train.shape

((3000, 28, 28), (3000,))

### 5. Train the neural network

In [5]:
net.fit(X_train, y_train, epochs=5, batch_size=64, shuffle=True)

Training on 3000 samples:
Epoch    1 of 5    	 Average Error = 0.783443 	 Average Accuracy = 78.00%                                   
Epoch    2 of 5    	 Average Error = 0.291126 	 Average Accuracy = 92.43%                                   
Epoch    3 of 5    	 Average Error = 0.188088 	 Average Accuracy = 95.37%                                   
Epoch    4 of 5    	 Average Error = 0.135816 	 Average Accuracy = 96.50%                                   
Epoch    5 of 5    	 Average Error = 0.105282 	 Average Accuracy = 97.63%                                   
Training time : 00 hours, 00 minutes, 39 seconds


### 6. Make predictions on the test samples

In [6]:
# Prediction on the 10,000 test samples:
y_pred = net.predict(X_test, to="labels")
# Shows the first 10 predicted labels vs the true labels:
y_pred[:10], y_test[:10]

(array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9]),
 array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], dtype=uint8))

### 7. Evaluate the accuracy score

In [7]:
print(f"Accuracy score on the test set: {accuracy_score(y_test, y_pred):.2%}")

Accuracy score on the test set: 95.67%


### 8. Build a confusion matrix

In [8]:
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion matrix:\n{cm}")

Confusion matrix:
[[ 966    1    0    0    1    1    6    1    4    0]
 [   0 1125    3    1    0    0    3    1    2    0]
 [  11    5  951    4   14    0    3   22   19    3]
 [   4    0    3  943    1   23    1   13   17    5]
 [   0    3    1    0  949    0    4    2    1   22]
 [   9    1    0    5    0  854    4    2   13    4]
 [  11    5    1    1    6    5  926    0    3    0]
 [   1    5   18    4    3    0    0  977    3   17]
 [   8    3    4    5    6    4    6    6  912   20]
 [   7    7    0    3    7    4    1   12    4  964]]


### 9. Displays a classification report

In [9]:
print(classification_report(cm, formatted=True))

┌───────────────────┬───────────┬───────────┬───────────┬───────────┐
│       Class       │ Precision │   Recall  │  F1-Score │  Support  │
├───────────────────┼───────────┼───────────┼───────────┼───────────┤
│         0         │  94.99%   │  98.57%   │  96.75%   │    980    │
│         1         │  97.40%   │  99.12%   │  98.25%   │   1135    │
│         2         │  96.94%   │  92.15%   │  94.49%   │   1032    │
│         3         │  97.62%   │  93.37%   │  95.45%   │   1010    │
│         4         │  96.15%   │  96.64%   │  96.39%   │    982    │
│         5         │  95.85%   │  95.74%   │  95.79%   │    892    │
│         6         │  97.06%   │  96.66%   │  96.86%   │    958    │
│         7         │  94.31%   │  95.04%   │  94.67%   │   1028    │
│         8         │  93.25%   │  93.63%   │  93.44%   │    974    │
│         9         │  93.14%   │  95.54%   │  94.32%   │   1009    │
└───────────────────┴───────────┴───────────┴───────────┴───────────┘
